In [7]:
# First cell - imports
import pandas as pd
import numpy as np
import re
from datetime import datetime

# Second cell - helper functions for processing different fields
def process_years_experience(years):
    if pd.isna(years):
        return np.nan
    
    years_str = str(years).lower()
    
    # Handle "Less than 1 year" or "Less than a year"
    if 'less than' in years_str:
        return 0
    
    # Handle ranges like "1-2 years", "3 to 5 years", etc.
    if '-' in years_str or ' to ' in years_str:
        # Replace "to" with "-" for consistency
        years_str = years_str.replace(' to ', '-')
        
        # Extract numbers from the range
        numbers = re.findall(r'\d+', years_str)
        if len(numbers) >= 2:
            lower = int(numbers[0])
            upper = int(numbers[1])
            # Return a random integer in the range
            return np.random.randint(lower, upper + 1)
    
    # Handle "More than X years" or "X or more years"
    if 'more than' in years_str or 'or more' in years_str:
        numbers = re.findall(r'\d+', years_str)
        if numbers:
            return int(numbers[0])
    
    # Try to extract a single number
    numbers = re.findall(r'\d+', years_str)
    if numbers:
        return int(numbers[0])
    
    return np.nan

def standardize_education(education):
    if pd.isna(education):
        return 'Bachelor'  # Default to Bachelor for missing values
    
    education = str(education).lower()
    
    if any(term in education for term in ['high school', 'secondary', 'elementary', 'primary', 'some college', 'associate', 'a.a.', 'a.s.']):
        return 'High School'
    elif any(term in education for term in ['bachelor', 'b.a.', 'b.s.', 'b.eng']):
        return 'Bachelor'
    elif any(term in education for term in ['master', 'm.a.', 'm.s.', 'm.eng', 'mba']):
        return 'Master'
    elif any(term in education for term in ['ph.d', 'doctoral', 'doctorate', 'ed.d']):
        return 'PhD'
    else:
        return 'Bachelor'  # Default to Bachelor for other cases

def standardize_job_role(job_role):
    if pd.isna(job_role):
        return np.nan
    
    job_role = str(job_role).lower()
    
    # Define categories
    categories = {
        'Back-end Developer': ['back-end', 'backend', 'back end'],
        'Front-end Developer': ['front-end', 'frontend', 'front end'],
        'Full-stack Developer': ['full-stack', 'fullstack', 'full stack'],
        'Mobile Developer': ['mobile'],
        'Game Developer': ['game', 'graphics'],
        'Embedded Engineer': ['embedded'],
        'Data Scientist': ['data scientist', 'machine learning', 'ai', 'artificial intelligence', 'data or business analyst', 'data analyst', 'business analyst', 'engineer, data'],
        'DevOps': ['devops', 'cloud', 'network', 'security', 'system', 'reliability', 'administrator', 'ops']
    }
    
    roles = []
    
    for category, keywords in categories.items():
        if any(keyword in job_role for keyword in keywords):
            roles.append(category)
    
    if not roles:
        return 'Other'
    
    return ';'.join(roles)

def standardize_gender(gender):
    if pd.isna(gender):
        return np.nan
    
    gender = str(gender).lower()
    
    if any(term in gender for term in ['male', 'man', 'men']):
        return 'Male'
    elif any(term in gender for term in ['female', 'woman', 'women']):
        return 'Female'
    else:
        return 'Other'

def process_age(age):
    if pd.isna(age):
        return np.nan
    
    age_str = str(age).lower()
    
    # Handle ranges like "25-34 years old"
    if '-' in age_str:
        numbers = re.findall(r'\d+', age_str)
        if len(numbers) >= 2:
            lower = int(numbers[0])
            upper = int(numbers[1])
            # Return a random integer in the range
            return np.random.randint(lower, upper + 1)
    
    # Handle "Under X years old"
    if 'under' in age_str:
        numbers = re.findall(r'\d+', age_str)
        if numbers:
            upper = int(numbers[0])
            return np.random.randint(upper - 5, upper)
    
    # Handle "X years or older"
    if 'or older' in age_str:
        numbers = re.findall(r'\d+', age_str)
        if numbers:
            lower = int(numbers[0])
            return np.random.randint(lower, lower + 10)
    
    # Try to extract a single number
    numbers = re.findall(r'\d+', age_str)
    if numbers:
        return int(numbers[0])
    
    return np.nan

# Third cell - functions for standardizing compensation and currency
def standardize_comp_freq(df):
    # Create a copy of the dataframe
    result_df = df.copy()
    
    # Define conversion factors to monthly
    conversion_factors = {
        'yearly': 1/12,
        'annual': 1/12,
        'weekly': 4,
        'daily': 20,
        'hourly': 160
    }
    
    # Convert CompFreq to lowercase for case-insensitive comparison
    result_df['CompFreq'] = result_df['CompFreq'].astype(str).str.lower()
    
    # Apply conversion factors
    for freq, factor in conversion_factors.items():
        mask = result_df['CompFreq'].str.contains(freq, na=False)
        result_df.loc[mask, 'CompTotal'] = result_df.loc[mask, 'CompTotal'] * factor
    
    # Set all CompFreq to Monthly
    result_df['CompFreq'] = 'Monthly'
    
    return result_df

def convert_currency_to_vnd(df):
    # Create a copy of the dataframe
    result_df = df.copy()
    
    # Define conversion rates to VND (approximate as of 2023)
    # These rates should be updated with more accurate or time-appropriate values
    conversion_rates = {
        'usd': 23000,  # US Dollar
        'eur': 25000,  # Euro
        'gbp': 29000,  # British Pound
        'sgd': 17000,  # Singapore Dollar
        'jpy': 160,    # Japanese Yen
        'aud': 15000,  # Australian Dollar
        'cad': 17000,  # Canadian Dollar
        'inr': 280,    # Indian Rupee
        'cny': 3200,   # Chinese Yuan
        'aed': 6300    # UAE Dirham
    }
    
    # Convert Currency to lowercase for case-insensitive comparison
    result_df['Currency'] = result_df['Currency'].astype(str).str.lower()
    
    # Apply conversion rates based on currency
    for currency, rate in conversion_rates.items():
        mask = result_df['Currency'].str.contains(currency, na=False)
        result_df.loc[mask, 'CompTotal'] = result_df.loc[mask, 'CompTotal'] * rate
    
    # Special case: If CompTotal is less than 5,000,000, assume it's in USD and convert
    small_values_mask = (result_df['CompTotal'] < 5000000) & (~result_df['Currency'].str.contains('vnd', na=False))
    result_df.loc[small_values_mask, 'CompTotal'] = result_df.loc[small_values_mask, 'CompTotal'] * conversion_rates['usd']
    
    # Set all Currency to VND
    result_df['Currency'] = 'VND'
    
    return result_df

def standardize_gender(gender):
    if pd.isna(gender):
        return 'Other'  # Convert NaN to 'Other'
    
    gender = str(gender).lower()
    
    if any(term in gender for term in ['male', 'man', 'men']):
        return 'Male'
    elif any(term in gender for term in ['female', 'woman', 'women']):
        return 'Female'
    else:
        return 'Other'

def process_salary_data(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Create a copy to avoid modifying the original dataframe
    processed_df = df.copy()
    
    # Standardize column names if needed
    if 'YearsExperience' not in processed_df.columns and 'YearsCodePro' in processed_df.columns:
        processed_df.rename(columns={'YearsCodePro': 'YearsExperience'}, inplace=True)
    
    # Process YearsExperience to integers
    processed_df['YearsExperience'] = processed_df['YearsExperience'].apply(process_years_experience)
    
    # Standardize Education
    processed_df['Education'] = processed_df['Education'].apply(standardize_education)
    
    # Standardize JobRole
    processed_df['JobRole'] = processed_df['JobRole'].apply(standardize_job_role)
    
    # Standardize Gender
    processed_df['Gender'] = processed_df['Gender'].apply(standardize_gender)
    
    # Process Age to integers
    processed_df['Age'] = processed_df['Age'].apply(process_age)
    
    # Fill missing Age values with formula: 22 + YearsExperience
    age_missing_mask = processed_df['Age'].isna()
    processed_df.loc[age_missing_mask, 'Age'] = 22 + processed_df.loc[age_missing_mask, 'YearsExperience']
    
    # Standardize CompFreq to monthly
    processed_df = standardize_comp_freq(processed_df)
    
    # Convert all currencies to VND
    processed_df = convert_currency_to_vnd(processed_df)
    
    # Drop rows with missing CompTotal after processing
    processed_df = processed_df.dropna(subset=['CompTotal'])
    
    # Select and reorder columns
    columns_order = ['YearsExperience', 'Education', 'JobRole', 'OrgSize', 'Location', 
                     'CompTotal', 'CompFreq', 'Currency', 'LanguageHaveWorkedWith', 'Age', 'Gender']
    
    # Ensure all required columns exist
    for col in columns_order:
        if col not in processed_df.columns:
            processed_df[col] = np.nan
    
    return processed_df[columns_order]

In [8]:
# Fifth cell - execution
# Set the file path
file_path = "../processed_data/combined_salary_data_2017_2025.csv"

# Process the data
processed_data = process_salary_data(file_path)

# Save the processed data to a new CSV file
output_path = "../processed_data/processed_salary_data_standardized.csv"
processed_data.to_csv(output_path, index=False)

print("Data processing completed. Saved to", output_path)
print(f"Processed {len(processed_data)} rows of data.")

# Display sample of processed data
print("\nSample of processed data:")
processed_data.head()

Data processing completed. Saved to ../processed_data/processed_salary_data_standardized.csv
Processed 716 rows of data.

Sample of processed data:


,YearsExperience,Education,JobRole,OrgSize,Location,CompTotal,CompFreq,Currency,LanguageHaveWorkedWith,Age,Gender
0,0.0,High School,Other,100 to 499 employees,Vietnam,220800000.0,Monthly,VND,CoffeeScript; JavaScript; Rust,22.0,Male
1,7.0,Bachelor,Other,"1,000 to 4,999 employees",Vietnam,276000000.0,Monthly,VND,C#,29.0,Male
2,2.0,Bachelor,Mobile Developer,10 to 19 employees,Vietnam,331200000.0,Monthly,VND,C++; Objective-C; Swift,24.0,Other
3,10.0,Bachelor,DevOps,10 to 19 employees,Vietnam,920000000.0,Monthly,VND,NaN,32.0,Other
4,12.0,Bachelor,Embedded Engineer;Data Scientist;DevOps,20 to 99 employees,Vietnam,414000000.0,Monthly,VND,Clojure; Java; JavaScript; Python; Scala; SQL,34.0,Male


In [19]:
import os
import joblib
import numpy as np
import pandas as pd
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score

def get_model_parameters(version=None):
    """Get model parameters and metadata for a specific version"""
        
    try:
        version_path = os.path.join("C:/Users/a5155733/MCS/IS/Assignment/RF-VN-Employee-Salary-Predictor/predictor_system/model_versions", f'v{version}')
        
        # Load model first as it's essential
        model = joblib.load(os.path.join(version_path, 'salary_model.pkl'))
        
        # Get basic model parameters
        params = {
            'n_estimators': model.n_estimators,
            'max_depth': model.max_depth,
            'min_samples_split': model.min_samples_split,
            'min_samples_leaf': model.min_samples_leaf,
            'max_features': model.max_features,
            'random_state': model.random_state
        }
        
        # Try to load metadata, but don't fail if it doesn't exist
        try:
            metadata = joblib.load(os.path.join(version_path, 'metadata.pkl'))
            params.update({
                'version': version,
                'created_at': metadata.get('created_at'),
                'feature_names': metadata.get('feature_names'),
                'metrics': metadata.get('metrics'),  # Ensure metrics are included
                'uses_engineered_features': metadata.get('uses_engineered_features', False),
                'is_log_transformed': metadata.get('is_log_transformed', False)
            })
        except Exception:
            params['version'] = version
        
        return params
        
    except Exception as e:
        print(f"Error loading model parameters: {e}")
        return None


model12 = get_model_parameters("13")

In [20]:
model12

{'n_estimators': 50,
 'max_depth': 21,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'random_state': 42,
 'version': '13',
 'created_at': '2025-11-28T14:26:17.832617',
 'feature_names': ['Education',
  'JobRole',
  'OrgSize',
  'CompFreq',
  'Currency',
  'LanguageHaveWorkedWith',
  'Gender'],
 'metrics': {'best_score': -0.670402139158815,
  'mse': 2653835447093353.0,
  'rmse': 51515390.39057506,
  'r2': 0.13823445798428136,
  'mae': 17465409.403879743,
  'mape': 53.62151625121911,
  'tuning_method': 'grid',
  'cv_folds': 5,
  'scoring': 'neg_root_mean_squared_error',
  'engineered_features': True},
 'uses_engineered_features': True,
 'is_log_transformed': True}